In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
from utils import load_pickle

pickle_file = 'notMNIST.pickle'
image_size = 28
num_labels = 10

dic = load_pickle(pickle_file)

['train_labels', 'valid_labels', 'test_labels', 'test_dataset', 'train_dataset', 'valid_dataset']
('train set', (200000, 28, 28), (200000,))
('valid set', (10000, 28, 28), (10000,))
('test set', (10000, 28, 28), (10000,))


In [20]:
from IPython.display import Latex

softmax = r"$S(x) = \frac{e^{x_i}}{\sum(e^{x_i})}$"
print ('softmax:')
Latex(softmax)

softmax:
cross entropy
notes:
L: one-hot labels
S: softmax


<IPython.core.display.Latex object>

In [19]:
print ('cross entropy')
print ('notes:\nL: one-hot labels\nS: softmax')
cross = r"$D =(S,L) = - \sum_i L_i log(S_i)$"
Latex(cross)

cross entropy
notes:
L: one-hot labels
S: softmax


<IPython.core.display.Latex object>

In [24]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(dic['train_dataset'][:train_subset, :])
  tf_train_labels = tf.constant(dic['train_labels'][:train_subset])
  tf_valid_dataset = tf.constant(dic['valid_dataset'])
  tf_test_dataset = tf.constant(dic['test_dataset'])

  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [26]:
num_steps = 801


def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) /
            predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases.
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            for 
            print('Training accuracy: %.1f%%' %
                  accuracy(predictions, dic['train_labels'][:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' %
                  accuracy(valid_prediction.eval(), dic['valid_labels']))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(),
                                             dic['test_labels']))

Initialized
Loss at step 0: 15.004263
Training accuracy: 15.5%
Validation accuracy: 17.7%
Loss at step 100: 2.184959
Training accuracy: 72.5%
Validation accuracy: 71.2%
Loss at step 200: 1.750292
Training accuracy: 75.6%
Validation accuracy: 73.8%
Loss at step 300: 1.528172
Training accuracy: 76.9%
Validation accuracy: 74.5%
Loss at step 400: 1.378100
Training accuracy: 77.8%
Validation accuracy: 74.9%
Loss at step 500: 1.265399
Training accuracy: 78.4%
Validation accuracy: 75.0%
Loss at step 600: 1.176330
Training accuracy: 79.1%
Validation accuracy: 75.2%
Loss at step 700: 1.103579
Training accuracy: 79.5%
Validation accuracy: 75.3%
Loss at step 800: 1.042690
Training accuracy: 80.0%
Validation accuracy: 75.4%
Test accuracy: 83.1%


# Problem
Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.